In [2]:
%reset -f
import random
from Basketball_Dataset_rnn import Dataset
import pandas as pd
from collections import deque
import numpy as np
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
SEQ_LEN = 3

In [4]:
dataset_20150320 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150320.csv', "20150320", home_visit = 1, save = True)
dataset_20141114 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20141114.csv', "20141114", home_visit = 1)
dataset_20141118 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20141118.csv', "20141118", home_visit = 1)
dataset_20141121 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20141121.csv', "20141121", home_visit = 1)
dataset_20141122 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20141122.csv', "20141122", home_visit = 1)
dataset_20141126 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20141126.csv', "20141126", home_visit = 1)
dataset_20141130 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20141130.csv', "20141130", home_visit = 1)
dataset_20141215 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20141215.csv', "20141215", home_visit = 1)
dataset_20141229 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20141229.csv', "20141229", home_visit = 1)
dataset_20141231 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20141231.csv', "20141231", home_visit = 1)
dataset_20150103 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150103.csv', "20150103", home_visit = 1)
dataset_20150113 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150113.csv', "20150113", home_visit = 1)
dataset_20150117 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150117.csv', "20150117", home_visit = 0)
dataset_20150119 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150119.csv', "20150119", home_visit = 1)
dataset_20150125 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150125.csv', "20150125", home_visit = 0)
dataset_20150204 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150204.csv', "20150204", home_visit = 1)
dataset_20150207 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150207.csv', "20150207", home_visit = 1)
dataset_20150218 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150218.csv', "20150218", home_visit = 1)
dataset_20150221 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150221.csv', "20150221", home_visit = 1)
dataset_20150228 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150228.csv', "20150228", home_visit = 1)
dataset_20150304 = Dataset('/Users/xiewenge/Documents/Work/Data +/Basketball Project/Wenge_Program/24_dataset/dataset_20150304.csv', "20150304", home_visit = 1)


dataset20150320 is imported.
dataset20141114 is imported.
dataset20141118 is imported.
dataset20141121 is imported.
dataset20141122 is imported.
dataset20141126 is imported.
dataset20141130 is imported.
dataset20141215 is imported.
dataset20141229 is imported.
dataset20141231 is imported.
dataset20150103 is imported.
dataset20150113 is imported.
dataset20150117 is imported.
dataset20150119 is imported.
dataset20150125 is imported.
dataset20150204 is imported.
dataset20150207 is imported.
dataset20150218 is imported.
dataset20150221 is imported.
dataset20150228 is imported.
dataset20150304 is imported.


In [5]:
def build_model(training_X):
#     print(training_X).shape
    model = Sequential()
    model.add(LSTM(128, input_shape=((SEQ_LEN, 49)), return_sequences=True))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(BatchNormalization())

    model.add(LSTM(128))
    model.add(Dropout(0.2))
    model.add(BatchNormalization())

    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(6, activation='softmax'))
    return model

In [6]:
def cross_fit(datasets_tr_val, datasets_te, epoch = 50):
    chosen_models = []
    for i in range(len(datasets_tr_val) - 1):
#     for i in range(2):
        print("********************************************************************")
        printline = "cross vadation processing the " + str(i) + " time"
        print(printline)
        exculde_list = [i, i + 1]
        validation_sets = [datasets_tr_val[i], datasets_tr_val[i + 1]]
        training_sets = [elem for j, elem in enumerate(datasets_tr_val) if j not in exculde_list]
        unprocessed_training = pd.concat(training_sets)
        unprocessed_validation = pd.concat(validation_sets)
        training_set = make_sequence_set(unprocessed_training)
        validation_set = make_sequence_set(unprocessed_validation)
        training_X, training_y = feature_label_splitting(training_set)
        validation_X, validation_y = feature_label_splitting(validation_set)
        model = build_model(training_X)
        opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
        model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
        checkpoint_path = "RNN_cross/cross_" + str(i) + "/cp-{epoch:04d}.ckpt"
        cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, save_weights_only=True, verbose=1)
        history = model.fit(training_X, training_y, batch_size=64, epochs=epoch, validation_data=(validation_X, validation_y), callbacks=[cp_callback])
        printline = "The best model is model_" + str(history.history["val_acc"].index(max(history.history["val_acc"])) + 1) + " with the validation_acc of " + str(max(history.history["val_acc"]))
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(printline)
        best_model_path = best_model_path = "RNN_cross/cross_%(i)d/cp-%(epoch)04d.ckpt" % {"i": i, "epoch": history.history["val_acc"].index(max(history.history["val_acc"])) + 1} 
        chosen_models.append(best_model_path)
#         model.load_weights(best_model_path)
#         loss,acc = model.evaluate(validation_X, validation_y)
#         print("Restored model, accuracy: {:5.2f}%".format(100*acc))
    if len(datasets_te) > 1:
        unprocessed_testing = pd.concat(datasets_te)
    unprocessed_testing = datasets_te[0]
    testing_set = make_sequence_set(unprocessed_testing)
    testing_X, testing_y = feature_label_splitting(testing_set)
    model = build_model(testing_X)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    highest_test_acc = 0
    best_testing_model = None
    for chosen_model in chosen_models:
        model.load_weights(chosen_model)
        _,acc = model.evaluate(testing_X, testing_y)
#         print("Restored model, accuracy: {:5.2f}%".format(100*acc))
        if acc >= highest_test_acc:
            highest_test_acc = acc
            best_testing_model = chosen_model
    printline = "The best model is: " + best_testing_model + " with testing accuracy of " + str(highest_test_acc)
    print("----------------------------------------------------------------------------------------------------")
    print(printline)
    return best_testing_model

In [7]:
def make_sequence_set(dataframe):
    first = 0
    second = 1
    sequential_data = []
    while first < dataframe.shape[0] or second < dataframe.shape[0]:
        ready_stop = False
        while ((second < dataframe.shape[0]) and ((dataframe["shot.clock"].iloc[second] < 0.9999) or (dataframe["shot.clock"].iloc[second] > 0.9999 and ready_stop == False))):
            if dataframe["shot.clock"].iloc[second] < 0.9999:
                ready_stop = True
            second += 1
        start_ind = dataframe["shot.clock"].iloc[[first]].index[0]
        end_ind = dataframe["shot.clock"].iloc[[second - 2]].index[0]
        start_iloc_ind = first
        end_iloc_ind = second - 2
#         temp_frame = dataframe.iloc[start_iloc_ind: end_iloc_ind,:]
#         saveline = "temp/temp" + str(first) + ".csv"
#         temp_frame.to_csv(saveline)
        sequence = deque(maxlen=SEQ_LEN)
        for row in dataframe.iloc[start_iloc_ind: end_iloc_ind,:].values:
            sequence.append([n for n in row[:-1]])
            if len(sequence) == SEQ_LEN:
                sequential_data.append([np.array(sequence), row[-1]])
        first = second
        second += 1
    random.shuffle(sequential_data)
    return sequential_data


In [8]:
def feature_label_splitting(dataset):
    X = []
    y = []

    for seq, target in dataset:  # going over our new sequential data
        X.append(seq)  # X is the sequences
        y.append(target)  # y is the targets/labels (buys vs sell/notbuy)

    return np.array(X), y  # return X and y...and make X a numpy array!


In [9]:
# i = 0
# best_model_path = "RNN_cross/cross_%(i)d/cp-%(epoch)04d.ckpt" % {"i": i, "epoch": history.history["val_acc"].index(max(history.history["val_acc"])) + 1} 
# print(best_model_path)

In [10]:
# epoch = 9
# b = '%(language)s has %(#)03d quote types.' % {'language': "Python", "#": 2}
# a = '%(E)d' % {"E":epoch}
# print(a)

In [27]:
datasets_tr_val = [dataset_20141114.nn_ready, dataset_20141118.nn_ready, dataset_20141121.nn_ready, dataset_20141122.nn_ready, dataset_20141126.nn_ready, dataset_20141130.nn_ready, dataset_20141215.nn_ready, dataset_20141229.nn_ready, dataset_20141231.nn_ready, dataset_20150103.nn_ready, dataset_20150113.nn_ready, dataset_20150117.nn_ready, dataset_20150119.nn_ready, dataset_20150125.nn_ready, dataset_20150204.nn_ready, dataset_20150207.nn_ready, dataset_20150218.nn_ready, dataset_20150221.nn_ready, dataset_20150228.nn_ready, dataset_20150304.nn_ready]
datasets_te = [dataset_20150320.nn_ready]
best_testing_model = cross_fit(datasets_tr_val, datasets_te, epoch = 50)

********************************************************************
cross vadation processing the 0 time
Train on 19368 samples, validate on 2083 samples
Epoch 1/50
19200/19368 [============================>.] - ETA: 0s - loss: 1.0231 - acc: 0.6077
Epoch 00001: saving model to RNN_cross/cross_0/cp-0001.ckpt
19368/19368 [==============================] - 16s 830us/sample - loss: 1.0201 - acc: 0.6085 - val_loss: 0.7927 - val_acc: 0.7417
Epoch 2/50
19264/19368 [============================>.] - ETA: 0s - loss: 0.6237 - acc: 0.7657
Epoch 00002: saving model to RNN_cross/cross_0/cp-0002.ckpt
19368/19368 [==============================] - 5s 235us/sample - loss: 0.6239 - acc: 0.7654 - val_loss: 0.5986 - val_acc: 0.7897
Epoch 3/50
19264/19368 [============================>.] - ETA: 0s - loss: 0.5946 - acc: 0.7718
Epoch 00003: saving model to RNN_cross/cross_0/cp-0003.ckpt
19368/19368 [==============================] - 4s 232us/sample - loss: 0.5940 - acc: 0.7720 - val_loss: 0.5778 - val_acc:

Epoch 7/50
19072/19213 [============================>.] - ETA: 0s - loss: 0.5531 - acc: 0.7760
Epoch 00007: saving model to RNN_cross/cross_1/cp-0007.ckpt
19213/19213 [==============================] - 5s 235us/sample - loss: 0.5522 - acc: 0.7765 - val_loss: 0.5602 - val_acc: 0.7878
Epoch 8/50
19008/19213 [============================>.] - ETA: 0s - loss: 0.5468 - acc: 0.7755
Epoch 00008: saving model to RNN_cross/cross_1/cp-0008.ckpt
19213/19213 [==============================] - 4s 233us/sample - loss: 0.5475 - acc: 0.7757 - val_loss: 0.5463 - val_acc: 0.7851
Epoch 9/50
19008/19213 [============================>.] - ETA: 0s - loss: 0.5447 - acc: 0.7766
Epoch 00009: saving model to RNN_cross/cross_1/cp-0009.ckpt
19213/19213 [==============================] - 5s 238us/sample - loss: 0.5442 - acc: 0.7766 - val_loss: 0.5437 - val_acc: 0.7869
Epoch 10/50
19200/19213 [============================>.] - ETA: 0s - loss: 0.5413 - acc: 0.7766
Epoch 00010: saving model to RNN_cross/cross_1/cp-00

Epoch 36/50
19008/19213 [============================>.] - ETA: 0s - loss: 0.4273 - acc: 0.8085
Epoch 00036: saving model to RNN_cross/cross_1/cp-0036.ckpt
19213/19213 [==============================] - 4s 234us/sample - loss: 0.4274 - acc: 0.8085 - val_loss: 0.6007 - val_acc: 0.7842
Epoch 37/50
19008/19213 [============================>.] - ETA: 0s - loss: 0.4286 - acc: 0.8080
Epoch 00037: saving model to RNN_cross/cross_1/cp-0037.ckpt
19213/19213 [==============================] - 4s 232us/sample - loss: 0.4285 - acc: 0.8082 - val_loss: 0.5873 - val_acc: 0.7748
Epoch 38/50
19008/19213 [============================>.] - ETA: 0s - loss: 0.4257 - acc: 0.8121
Epoch 00038: saving model to RNN_cross/cross_1/cp-0038.ckpt
19213/19213 [==============================] - 4s 233us/sample - loss: 0.4253 - acc: 0.8125 - val_loss: 0.5869 - val_acc: 0.7828
Epoch 39/50
19008/19213 [============================>.] - ETA: 0s - loss: 0.4189 - acc: 0.8129
Epoch 00039: saving model to RNN_cross/cross_1/cp

Epoch 14/50
18944/19067 [============================>.] - ETA: 0s - loss: 0.5242 - acc: 0.7786
Epoch 00014: saving model to RNN_cross/cross_2/cp-0014.ckpt
19067/19067 [==============================] - 5s 238us/sample - loss: 0.5247 - acc: 0.7782 - val_loss: 0.5479 - val_acc: 0.7596
Epoch 15/50
19008/19067 [============================>.] - ETA: 0s - loss: 0.5187 - acc: 0.7801
Epoch 00015: saving model to RNN_cross/cross_2/cp-0015.ckpt
19067/19067 [==============================] - 5s 238us/sample - loss: 0.5185 - acc: 0.7802 - val_loss: 0.5534 - val_acc: 0.7668
Epoch 16/50
18944/19067 [============================>.] - ETA: 0s - loss: 0.5117 - acc: 0.7825
Epoch 00016: saving model to RNN_cross/cross_2/cp-0016.ckpt
19067/19067 [==============================] - 4s 234us/sample - loss: 0.5117 - acc: 0.7826 - val_loss: 0.5529 - val_acc: 0.7689
Epoch 17/50
19008/19067 [============================>.] - ETA: 0s - loss: 0.5101 - acc: 0.7826
Epoch 00017: saving model to RNN_cross/cross_2/cp

Epoch 43/50
19008/19067 [============================>.] - ETA: 0s - loss: 0.3955 - acc: 0.8274
Epoch 00043: saving model to RNN_cross/cross_2/cp-0043.ckpt
19067/19067 [==============================] - 5s 241us/sample - loss: 0.3956 - acc: 0.8275 - val_loss: 0.6634 - val_acc: 0.7504
Epoch 44/50
18944/19067 [============================>.] - ETA: 0s - loss: 0.3930 - acc: 0.8284
Epoch 00044: saving model to RNN_cross/cross_2/cp-0044.ckpt
19067/19067 [==============================] - 5s 242us/sample - loss: 0.3925 - acc: 0.8286 - val_loss: 0.6789 - val_acc: 0.7559
Epoch 45/50
19008/19067 [============================>.] - ETA: 0s - loss: 0.3808 - acc: 0.8335
Epoch 00045: saving model to RNN_cross/cross_2/cp-0045.ckpt
19067/19067 [==============================] - 5s 237us/sample - loss: 0.3809 - acc: 0.8334 - val_loss: 0.6947 - val_acc: 0.7525
Epoch 46/50
19008/19067 [============================>.] - ETA: 0s - loss: 0.3815 - acc: 0.8321
Epoch 00046: saving model to RNN_cross/cross_2/cp

...

Epoch 25/50
18880/19073 [============================>.] - ETA: 0s - loss: 0.4819 - acc: 0.7864
Epoch 00025: saving model to RNN_cross/cross_16/cp-0025.ckpt
19073/19073 [==============================] - 5s 252us/sample - loss: 0.4819 - acc: 0.7867 - val_loss: 0.5716 - val_acc: 0.7973
Epoch 26/50
18944/19073 [============================>.] - ETA: 0s - loss: 0.4771 - acc: 0.7894
Epoch 00026: saving model to RNN_cross/cross_16/cp-0026.ckpt
19073/19073 [==============================] - 5s 250us/sample - loss: 0.4768 - acc: 0.7897 - val_loss: 0.5850 - val_acc: 0.7855
Epoch 27/50
19008/19073 [============================>.] - ETA: 0s - loss: 0.4711 - acc: 0.7907
Epoch 00027: saving model to RNN_cross/cross_16/cp-0027.ckpt
19073/19073 [==============================] - 5s 252us/sample - loss: 0.4714 - acc: 0.7906 - val_loss: 0.5831 - val_acc: 0.7923
Epoch 28/50
18880/19073 [============================>.] - ETA: 0s - loss: 0.4660 - acc: 0.7927
Epoch 00028: saving model to RNN_cross/cross_1

Epoch 3/50
18944/19011 [============================>.] - ETA: 0s - loss: 0.5975 - acc: 0.7748
Epoch 00003: saving model to RNN_cross/cross_17/cp-0003.ckpt
19011/19011 [==============================] - 5s 252us/sample - loss: 0.5977 - acc: 0.7747 - val_loss: 0.5711 - val_acc: 0.7590
Epoch 4/50
19008/19011 [============================>.] - ETA: 0s - loss: 0.5785 - acc: 0.7777
Epoch 00004: saving model to RNN_cross/cross_17/cp-0004.ckpt
19011/19011 [==============================] - 5s 253us/sample - loss: 0.5786 - acc: 0.7777 - val_loss: 0.5492 - val_acc: 0.7594
Epoch 5/50
18816/19011 [============================>.] - ETA: 0s - loss: 0.5679 - acc: 0.7776
Epoch 00005: saving model to RNN_cross/cross_17/cp-0005.ckpt
19011/19011 [==============================] - 5s 255us/sample - loss: 0.5671 - acc: 0.7780 - val_loss: 0.5468 - val_acc: 0.7602
Epoch 6/50
18944/19011 [============================>.] - ETA: 0s - loss: 0.5596 - acc: 0.7785
Epoch 00006: saving model to RNN_cross/cross_17/cp

Epoch 32/50
18816/19011 [============================>.] - ETA: 0s - loss: 0.4582 - acc: 0.8013
Epoch 00032: saving model to RNN_cross/cross_17/cp-0032.ckpt
19011/19011 [==============================] - 5s 253us/sample - loss: 0.4585 - acc: 0.8009 - val_loss: 0.5511 - val_acc: 0.7586
Epoch 33/50
18880/19011 [============================>.] - ETA: 0s - loss: 0.4505 - acc: 0.8038
Epoch 00033: saving model to RNN_cross/cross_17/cp-0033.ckpt
19011/19011 [==============================] - 5s 267us/sample - loss: 0.4515 - acc: 0.8035 - val_loss: 0.5606 - val_acc: 0.7602
Epoch 34/50
18944/19011 [============================>.] - ETA: 0s - loss: 0.4550 - acc: 0.8037
Epoch 00034: saving model to RNN_cross/cross_17/cp-0034.ckpt
19011/19011 [==============================] - 5s 267us/sample - loss: 0.4549 - acc: 0.8037 - val_loss: 0.5609 - val_acc: 0.7582
Epoch 35/50
18816/19011 [============================>.] - ETA: 0s - loss: 0.4456 - acc: 0.8051
Epoch 00035: saving model to RNN_cross/cross_1

Epoch 10/50
18752/18872 [============================>.] - ETA: 1s - loss: 0.5421 - acc: 0.7794
Epoch 00010: saving model to RNN_cross/cross_18/cp-0010.ckpt
18872/18872 [==============================] - 228s 12ms/sample - loss: 0.5431 - acc: 0.7789 - val_loss: 0.5289 - val_acc: 0.7759
Epoch 11/50
18752/18872 [============================>.] - ETA: 0s - loss: 0.5410 - acc: 0.7789
Epoch 00011: saving model to RNN_cross/cross_18/cp-0011.ckpt
18872/18872 [==============================] - 5s 258us/sample - loss: 0.5408 - acc: 0.7787 - val_loss: 0.5059 - val_acc: 0.7774
Epoch 12/50
18688/18872 [============================>.] - ETA: 3s - loss: 0.5319 - acc: 0.7792
Epoch 00012: saving model to RNN_cross/cross_18/cp-0012.ckpt
18872/18872 [==============================] - 332s 18ms/sample - loss: 0.5318 - acc: 0.7794 - val_loss: 0.5394 - val_acc: 0.7763
Epoch 13/50
18816/18872 [============================>.] - ETA: 0s - loss: 0.5322 - acc: 0.7806
Epoch 00013: saving model to RNN_cross/cross

Epoch 39/50
18624/18872 [============================>.] - ETA: 0s - loss: 0.4066 - acc: 0.8192
Epoch 00039: saving model to RNN_cross/cross_18/cp-0039.ckpt
18872/18872 [==============================] - 5s 267us/sample - loss: 0.4074 - acc: 0.8189 - val_loss: 0.5540 - val_acc: 0.7689
Epoch 40/50
18816/18872 [============================>.] - ETA: 0s - loss: 0.4043 - acc: 0.8199
Epoch 00040: saving model to RNN_cross/cross_18/cp-0040.ckpt
18872/18872 [==============================] - 5s 253us/sample - loss: 0.4041 - acc: 0.8199 - val_loss: 0.5749 - val_acc: 0.7662
Epoch 41/50
18816/18872 [============================>.] - ETA: 0s - loss: 0.3986 - acc: 0.8236
Epoch 00041: saving model to RNN_cross/cross_18/cp-0041.ckpt
18872/18872 [==============================] - 5s 259us/sample - loss: 0.3985 - acc: 0.8234 - val_loss: 0.5677 - val_acc: 0.7790
Epoch 42/50
18752/18872 [============================>.] - ETA: 0s - loss: 0.3974 - acc: 0.8244
Epoch 00042: saving model to RNN_cross/cross_1

In [28]:
dataset_20150320.nn_ready

,game.clock,shot.clock,event.descrip_0,event.descrip_1,event.descrip_2,event.descrip_3,event.descrip_4,event.descrip_5,event.descrip_6,position_0,...,p.poss_2,p.poss_3,p.poss_4,p.poss_5,p.poss_6,p.poss_7,p.poss_8,p.poss_9,score_difference,label
34,0.976522,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,2
35,0.975776,1.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,0
36,0.975640,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
37,0.974971,0.994213,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
38,0.974360,0.973380,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
39,0.973487,0.943287,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
40,0.972571,0.912037,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,2
41,0.972105,0.895544,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,0
42,0.971766,0.883970,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,1
43,0.970985,0.857350,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-2,2


In [11]:
def make_sequence_test(dataframe, row_num, SEQ_LEN):
    for i in range(SEQ_LEN - 1):
        if dataframe["shot.clock"].iloc[row_num - i - 1] < dataframe["shot.clock"].iloc[row_num - i]:
            print("This frame is too early in a shot clock period")
            return np.array([])
    sequence = deque(maxlen=SEQ_LEN)
    for row in dataframe.iloc[row_num - SEQ_LEN + 1: row_num + 1,:].values:
        sequence.append([n for n in row[:-1]])
    return np.array([sequence])
    

In [12]:
def predict(dataframe, row_num, chosen_model, SEQ_LEN, verbose = True):
    if(row_num < SEQ_LEN - 1):
        if verbose == True:
            print("You should give a sequence with length at least of ", SEQ_LEN)
        return 0, -1
    testing_X =make_sequence_test(dataframe, row_num, SEQ_LEN)
    if testing_X.shape[0] == 0:
        return 0, -1
    label = ['touch', 'dribble', 'pass', "shot", "rebound", "turnover"]
# #     temp = np.zeros((2,sample.shape[0]))
# #     temp[0] = sample.values
# #     temp[1] = sample.values
# #     test_df = pd.DataFrame(temp)
#     testing_set = make_sequence_set(sample)
# #     print(testing_set)
#     testing_X, testing_y = feature_label_splitting(testing_set)

    model = build_model(testing_X)
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.load_weights(chosen_model)
    probs = model.predict(testing_X)
    i = 0
    for prob in probs[0]:
        printline = "There are " + str(round((prob * 100),2)) + "% probability that the ball-holder's next move is " + label[i]
        if verbose == True:
            print(printline)
        i += 1
    prediction = list(probs[0]).index(max(list(probs[0])))
    return probs, prediction

In [13]:
row_num = 1
dataset = dataset_20150320.nn_ready
label_list = ['touch', 'dribble', 'pass', "shot", "rebound", "turnover"]
prediction_list, prediction_label = predict(dataset, row_num, "RNN_cross/cross_16/cp-0031.ckpt", SEQ_LEN)
label = ['touch', 'dribble', 'pass', "shot", "rebound", "turnover"]
print("The actual label is ", label_list[int(dataset.iloc[row_num]["label"])])

You should give a sequence with length at least of  3
The actual label is  touch


In [14]:
sum = 0
for element in dataset["label"]:
    if element == 1:
        sum += 1
print(sum / dataset.shape[0])

0.44872677219545765


In [15]:
def erro_analyze(dataset, chosen_model, label_list):
    prediction_list = []
    erro_list = [0] * 6
    total_row_num = dataset.shape[0]
    for row_num in range(0, dataset.shape[0]):
        printline = "The total row number is " + str(total_row_num) + ", and now we are at row " + str(row_num)
        print(printline)
        _, prediction_label = predict(dataset, row_num, chosen_model, SEQ_LEN, verbose = False)
        prediction_list.append(prediction_label)
        if prediction_label == -1:
            continue
        actual_label = dataset.iloc[row_num]["label"]
        if actual_label != prediction_label:
            erro_list[int(actual_label)] += 1
    return erro_list, prediction_list

In [16]:
erro_list, prediction_list = erro_analyze(dataset, 'RNN_cross/cross_16/cp-0031.ckpt', label_list)
for erro in erro_list:
    printline = "The error rate of " + label_list[erro_list.index[erro]] + "is " + str(erro / sum(erro_list))
    print(printline)

W0726 11:45:54.902277 4519372224 deprecation.py:506] From /anaconda3/lib/python3.7/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1628: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


The total row number is 1453, and now we are at row 0
The total row number is 1453, and now we are at row 1
The total row number is 1453, and now we are at row 2
The total row number is 1453, and now we are at row 3


W0726 11:45:56.993216 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0726 11:45:56.994262 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0726 11:45:56.994710 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0726 11:45:56.995178 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0726 11:45:56.995798 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0726 11:45:56.996459 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.gamma
W0726 11:45:56.997278 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.beta
W0726 11:45:56.997848 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-3.gamma
W0726 11:45:56.998531 4519372224 util.py:144

The total row number is 1453, and now we are at row 4


W0726 11:45:58.808427 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.iter
W0726 11:45:58.809494 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_1
W0726 11:45:58.810261 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.beta_2
W0726 11:45:58.810786 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.decay
W0726 11:45:58.811454 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer.learning_rate
W0726 11:45:58.812024 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.gamma
W0726 11:45:58.812699 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-1.beta
W0726 11:45:58.813232 4519372224 util.py:144] Unresolved object in checkpoint: (root).optimizer's state 'm' for (root).layer_with_weights-3.gamma
W0726 11:45:58.813789 4519372224 util.py:144

KeyboardInterrupt: 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
erro_analyze(dataset, 'RNN_cross/cross_16/cp-0031.ckpt', label_list)

In [ ]:
combined_datasets = pd.concat([dataset_20141114.nn_ready, dataset_20141118.nn_ready, dataset_20141121.nn_ready, dataset_20141122.nn_ready, dataset_20141126.nn_ready, dataset_20141130.nn_ready, dataset_20141215.nn_ready, dataset_20141229.nn_ready, dataset_20141231.nn_ready, dataset_20150103.nn_ready, dataset_20150113.nn_ready, dataset_20150117.nn_ready, dataset_20150119.nn_ready, dataset_20150125.nn_ready, dataset_20150204.nn_ready, dataset_20150207.nn_ready, dataset_20150218.nn_ready, dataset_20150221.nn_ready, dataset_20150228.nn_ready, dataset_20150304.nn_ready])    

In [ ]:
# combined_datasets = dataset_20141118.nn_ready 

In [ ]:
unprocessed_testing = dataset
testing_set = make_sequence_set(unprocessed_testing)
testing_X, testing_y = feature_label_splitting(testing_set)
model = build_model(testing_X)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
prediction_list = model

In [ ]:
feature_df = combined_datasets.iloc[:, :-1]
label_df = combined_datasets.iloc[:,[-1]]

In [ ]:
unprocessed_training = combined_datasets.iloc[: int(combined_datasets.shape[0] * 0.85),:]
unprocessed_validation = combined_datasets.iloc[int(combined_datasets.shape[0] * 0.85) + 1:,:]


In [ ]:
training_set = make_sequence_set(unprocessed_training)
validation_set = make_sequence_set(unprocessed_validation)

In [ ]:
training_X, training_y = feature_label_splitting(training_set)
validation_X, validation_y = feature_label_splitting(validation_set)

In [ ]:
print(training_X.shape[1:])

In [ ]:
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
NAME = f"{SEQ_LEN}-SEQ-{int(time.time())}"

In [ ]:
# model = Sequential()
# model.add(LSTM(128, input_shape=(training_X.shape[1:]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

# model.add(LSTM(128, return_sequences=True))
# model.add(Dropout(0.1))
# model.add(BatchNormalization())

# model.add(LSTM(128))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.2))

# model.add(Dense(6, activation='softmax'))

In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

In [ ]:
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

In [ ]:
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
checkpoint_path = "RNN/cp-{epoch:04d}.ckpt"
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [ ]:
# history = model.fit(
#     training_X, training_y,
#     batch_size=64,
#     epochs=20,
#     validation_data=(validation_X, validation_y)
# )
history = model.fit(
    training_X, training_y,
    batch_size=64,
    epochs=1,
    validation_data=(validation_X, validation_y),
    callbacks=[cp_callback]
)

In [ ]:
print(history.history["val_acc"])

In [ ]:
model.load_weights("RNN/cp-0022.ckpt")

In [ ]:
loss,acc = model.evaluate(validation_X, validation_y)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
unprocessed_validation_20141118 = dataset_20141118.nn_ready.iloc[:]
validation_20141118_set = make_sequence_set(unprocessed_validation_20141118)
validation_20141118_X, validation_20141118_y = feature_label_splitting(validation_20141118_set)
loss_20141118,acc_20141118 = model.evaluate(validation_20141118_X, validation_20141118_y)
print("Restored model, accuracy: {:5.2f}%".format(100*acc_20141118))


In [ ]:
unprocessed_validation_20150320 = dataset_20150320.nn_ready.iloc[:]
validation_20150320_set = make_sequence_set(unprocessed_validation_20150320)
validation_20150320_X, validation_20150320_y = feature_label_splitting(validation_20150320_set)
loss_20150320,acc_20150320 = model.evaluate(validation_20150320_X, validation_20150320_y)
print("Restored model, accuracy: {:5.2f}%".format(100*acc_20150320))


In [ ]:
prediction = model.predict(validation_X[:2])
print(prediction)

In [ ]:
print(validation_y[:2])

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0



In [ ]:
# Returns a short sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation=tf.keras.activations.relu, input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation=tf.keras.activations.softmax)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.sparse_categorical_crossentropy,
                metrics=['accuracy'])

  return model


# Create a basic model instance
model = create_model()
model.summary()

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # Save weights, every 5-epochs.
    period=5)


model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.


In [ ]:
model = create_model()

loss, acc = model.evaluate(test_images, test_labels)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
model.load_weights("training_2/cp-0005.ckpt")
loss,acc = model.evaluate(test_images, test_labels)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

In [ ]:
print(checkpoint_path)

In [ ]:
a = [1,2,3]
b = a.index(max(a))
print(b)

In [ ]:
for i in range(2,4):
    print(i)

In [ ]:
a = [0] * 5
print(a)

In [ ]:
print((np.array([])).shape[0])